<h1 align="center">Analisi della qualità del vino Vinho Verde</h1>
<h3 align="center">Stefano Brilli - Marzo 2019</h3>
<hr><hr>

<h2 align="center">Introduzione</h2>

<img src="img/wine.jpg" width="600">

In questo report saranno analizzati dati riguardanti il vino portoghese <b>Vinho Verde</b> nella sua variante rossa, allo scopo di predire se, in base ai valori osservati, si stia osservando un vino di bassa o alta qualità. Il dataset utilizzato è reperibile all'indirizzo http://archive.ics.uci.edu/ml/datasets/Wine+Quality.
Al fine di costruire un classificatore binario, il dataset è stato modificato per rendere la variabile di output <b>(quality)</b> una quantità binaria. In particolare, le osservazioni aventi un output minore od uguale a 6 sono stati etichettati con il <b>valore 0 (bassa qualità)</b> mentre quelli aventi un valore qualitativo di 7 o superiore sono stati etichettati con il <b>valore 1 (alta qualità)</b>.<br>
È chiaro come una mappatura di questo tipo ponga sia i vini con punteggio 6 che quelli con punteggio 10 al pari, nella categoria dei vini di alta qualità. D'altra parte, essa potrebbe permettere ai produttori di avere un iniziale responso della qualità del proprio vino, scartando in prima battuta quelli classificati come di bassa qualità ed andando successivamente ad analizzare nel dettaglio i restanti vini al fine di determinare una scala delle qualità.<br>
I valori di input sono stati calcolati per mezzo di test fisico-chimici, mentre l'output è stato determinato tramite analisi sensoristiche.
Gli attributi del dataset sono i seguenti:

<ul>
  <li>acidità fissa</li>
  <li>acidità volatile</li>
  <li>acido citrico</li>
  <li>zucchero residuo</li>
  <li>cloruri</li>
  <li>anidride solforosa libera</li>
  <li>anidride solforosa totale</li>
  <li>densità</li>
  <li>pH</li>
  <li>solfati</li>
  <li>alcool</li>
</ul>

<hr>
<h3 align="center">Strumenti utilizzati per l'analisi</h3>
L'intera analisi è stata condotta utilizzando il linguaggio <b>Python</b> e l'ambiente <b>Jupyter</b>.<br>
Le librerie utilizzate sono:
<ul>
    <li><b>pandas</b> (<a href="https://pandas.pydata.org/">https://pandas.pydata.org/</a>), libreria software per la manipolazione e l'analisi dei dati</li>
    <li><b>numpy</b> (<a href="http://www.numpy.org/">http://www.numpy.org/</a>), estensione open source del linguaggio di programmazione Python, che aggiunge supporto per vettori e matrici multidimensionali e di grandi dimensioni e con funzioni matematiche di alto livello con cui operare</li>
    <li><b>plotly</b> (<a href="https://plot.ly/feed/">https://plot.ly/feed/</a>), insieme di strumenti per l'analisi e la visualizzazione dei dati</li>
    <li><b>sklearn</b> (<a href="https://scikit-learn.org/stable/">https://scikit-learn.org/stable/</a>), libreria open source di apprendimento automatico per il linguaggio di programmazione Python</li>
</ul>    

<hr><hr>
<h2 align="center">Esplorazione e visualizzazione dei dati</h2>

Il dataset contiene <b>1599 righe</b> e <b>12 colonne</b> (11 di input + 1 di output). Il dataset non è stato alterato nel numero di osservazioni in quanto non risultavano valori mancanti.

In [6]:
# Tesina del corso Data Spaces
# Anno accademico 2018/2019
# Stefano Brilli, matricola s249914

import numpy as np
import pandas as pd


from scipy import interp


from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA as sklearnPCA

import copy

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)
from plotly.offline import plot, iplot
import plotly.plotly as py

from IPython.display import Math
import sys

from sklearn import svm
from warnings import simplefilter
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import (f1_score, recall_score, precision_score, roc_curve, roc_auc_score)
from sklearn.linear_model import LogisticRegression

import math

from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve

import warnings

from IPython.core.display import display, HTML 

display(HTML("<style>.container { width:100% !important; }</style>"))

warnings.filterwarnings('ignore')

simplefilter(action='ignore', category=FutureWarning)

np.set_printoptions(threshold=sys.maxsize)
# Questa funzione legge i dati dal file elimina le righe che contengono valori non numerici e mappa l'output sui valori 0/1
# Se quality <= 6 --> 0
# Altrimenti quality --> 1
def read_data():
#     path = '/home/stefano/Documenti/Politecnico/Magistrale/2 Anno/Data Spaces/tesina/wine_quality/'
    df = pd.read_csv('./winequality-red.csv', sep=';')  
    df = df[df.applymap(np.isreal).any(1)]
    df['quality'] = df['quality'].map(lambda x: 1 if x >= 7 else 0) 
    return df

In [7]:
df = read_data()
df.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,0
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,0
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,1
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,1
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,0


Osservando soltanto queste prime dieci righe possiamo notare uno sbilanciamento tra i vini di bassa qualità e quelli di alta qualità. Ovviamente il campione non può essere preso come rappresentativo dell'intero dataset. Nel grafico seguente viene quindi mostrata la distribuzione delle due etichette di output.

In [8]:
df.drop(columns=['quality']).describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000


In [9]:
colors = ['#d62728', '#2ca02c']
quality_values = {0: "Bassa", 1: "Alta"}

In [10]:
y = df["quality"].value_counts()

data = [go.Bar(x=[quality_values[x] for x in y.index], y=y.values, marker = dict(color = colors[:len(y.index)]))]
layout = go.Layout(
    title='Distribuzione della qualità',
    autosize=False,
    width=400,
    height=400,
    yaxis=dict(
        title='Numero di osservazioni',
    ),
    xaxis=dict(
        title='Qualità'
    ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='basic-bar3')

Come è possibile osservare i dati sono distrubuiti in maniera quasi bilanciata relativamente al valore di output e questo potrebbe significare che il modello di predizione sarà in grado di discriminare bene tra le due classi nella fase di classificazione di nuovi dati. <br>Nel dettaglio, i dati si distribuiscono come segue:
  
<ul>  
    <li><b>Bassa qualità</b> (1382 su 1599) = <b>86.43%</b></li>
    <li><b>Alta qualità</b> (217 su 1599) = <b>13.57%</b></li>
</ul>

<hr>
<h3 align="center">Distribuzione dei valori delle feature</h3>

Utilizzando i box plot è possibile osservare in che modo i valori delle feature sono distribuiti. Essi ci permettono di leggerne valore medio, massimo, minimo e rilevare eventuali outlier, ossia dati che non seguono il trend del gruppo a cui appartengono. Se presenti in numero abbastanza elevato, tali valori possono in alcuni casi rendere più complicata l'analisi, perchè il modello apprenderà pattern che si allontanano dal reale andamento dei dati in analisi.
I valori sono stati normalizzati al fine di apprezzare meglio la visualizzazione, in quanto alcune feature presentavano valori estremamente alti rendendo il grafico schiacciato verso il basso.

In [11]:
from plotly import tools
# Normalizzazione dei dati (tra 0 e 1)
X = df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X)
df_norm = pd.DataFrame(x_scaled)

def box_plot_by_feature(X):
    colors = ['#d62728', '#2ca02c']
    y_box_negative = X.loc[X["quality"] == 0]
    y_box_positive = X.loc[X["quality"] == 1]    
    data = []
    X = X.drop(columns=['quality'])
    i = 0
    traces_positions = [[1,1], [1,2], [1,3], [2,1], [2,2], [2,3], [3,1], [3,2], [3,3], [4,1], [4,2]]
    
    fig = tools.make_subplots(rows=4, cols=3, print_grid=False, \
                              subplot_titles=('Acidità fissa', 'Acidità volatile', 'Acido citrico', 'Zucchero residuo', "Cloruri", \
                                              "Diossido di solfuro libero", \
                                              "Diossido di solfuro totale", "Densità", "pH", "Solfati", "Alcol"))
    
    for column in X.columns:
        trace_0 = go.Box(
            name = "Bassa qualità",
            y = y_box_negative[column].tolist(),
            showlegend=False,
            marker = dict(color = colors[0])
        )        
        fig.append_trace(trace_0, traces_positions[i][0], traces_positions[i][1])        
        trace_1 = go.Box(
            name = "Alta qualità",
            y = y_box_positive[column].tolist(),
            showlegend=False,
            marker = dict(color = colors[1])
        )   
        fig.append_trace(trace_1, traces_positions[i][0], traces_positions[i][1])
        i += 1
    
    
    fig['layout'].update(height=1200, width=1200, title='Qualità rispetto alle feature')    
    iplot(fig)

# Costruzione delle strutture per rappresentare i box plot
data = []
for i in range(len(df.columns) - 1):
    data.append(go.Box(
        name = df.columns[i],
        y = df_norm[i].tolist()        
    ))
df_norm.columns = df.columns    
fig = go.Figure(data=data)
iplot(fig) 

Per ciascuna feature possiamo disegnare un box plot per vedere come esse si ditribuiscono a seconda della qualità misurata del vino. Se trovassimo feature i cui valori sono sovrapposti e che quindi il modello non potrebbe usare per discriminare la qualità del vino potremmo decidere di eliminarle, lasciando soltanto quelle feature che costituiscono una discriminante.

In [12]:
# Distribuzione dell'acidità fissa in base alla qualità del vino
box_plot_by_feature(copy.copy(df))

Alcune feature non sembrano essere una buona discriminate per la qualità del vino. Tra questi troviamo lo <b>zucchero residuo</b>, che a meno di alcuni outlier nei vini di bassa qualità risulta distribuito quasi allo stesso modo per entrambe le etichette. Allo stesso modo si comporta la feature <b>cloruri</b>, anch'essa non particolarmente discriminante della qualità. Nel proseguire l'analisi saranno quindi rimosse le suddette caratteristiche.

<hr>
<h3 align="center">Correlazione tra le feature</h3>

Il grafico sottostante riporta la correlazione che intercorre tre la feature. Una correlazione positiva indica che quando la prima feature cresce anche la seconda cresce. Una correlazione negativa indica una tendenza inversa all'aumento: quando la prima aumenta la seconda tende a diminuire.

In [13]:
df_ = copy.copy(df_norm).drop(columns=['quality'])
covariance_matrix = df_.corr()
covariance_matrix = covariance_matrix.values
x_ = []
y_ = []
z_ = []

for i in range(len(df_.columns)):
    z_.append(covariance_matrix[i,])
    x_.append(df_norm.columns[i])
    y_.append(df_norm.columns[i])

trace = go.Heatmap(z=z_,
                   x=x_,
                   y=y_)    
figure = go.Figure(data=[trace])
iplot(figure)

La maggior parte delle delle feature sono scarsamente correlate tra loro (è possibile notarlo per via della colorazione di tonalità azzurra che significa correlazione vicina allo zero) mentre altre sono correlate, sia direttamente che inversamente. Ad esempio, le feature <b>pH</b> e <b>citric acid</b> sono, come ci si potrebbe aspettare, inversamente correlate in quanto al crescere dell'acidità di una sostanza il pH misurato scende. Per questo motivo si potrebbe pensare di eliminare la feature relativa al pH.

In [14]:
# Rimozione delle feature che presentano valori troppo distribuiti, in accordo con il box plot precedente
df_norm = df_norm.drop(columns=['residual sugar', 'chlorides', 'pH'])

<hr><hr>
<h2 align="center">Analisi delle componenti principali (PCA)</h2>

L'analisi delle componenti principali (PCA, dall'inglese <b>Principal Components Analysis</b>) è una tecnica che nasce dalla necessità di migliorare le prestazioni degli algoritmi di analisi dei dati a fronte di un enorme incremento della numerosità di essi a disposizione degli analisti. Lo scopo della PCA è dunque quello di rilevare una possibile correlazione tra le diverse variabili di un dataset tale per cui la maggioranza della variabilità dell'informazione dipenda soltanto da esse. Riproiettando i dati lungo gli assi rilevati e scartando le direzioni povere di variabilità è possibile ridurre la dimensionalità del dataset, migliorando le prestazioni ed ottimizzando le risorse.
Per il calcolo delle componenti principali l'approccio è il seguente:
<ul>  
    <li>i dati vengono standardizzati</li>
    <li>si calcolano gli autovalori e gli autovettori dalla matrice di covarianza o da quella di correlazione, oppure si calcola la decomposizione a valore singolo</li>
    <li>si ordinano gli autovalori in ordine decrescente e si calcolano gli autovettori associati ad essi</li>
    <li>si costruisce la matrice W dei dati riproiettati utilizzando gli autovettori selezionati</li>
    <li>si trasforma il dataset originale tramite la matrice W per ottenere un sottospazio Y delle feature di dimensione pari al numero di autovettori utilizzati</li>
</ul>

Nel contesto di questo report sono state utilizzate le funzioni messe a disposizioni da Sklearn per il calcolo della PCA, le quali utilizzano il metodo della decomposizione a valore singolo (SVD) per trovare le componenti principali.

In [15]:
def doPCA(X_std, components, print_flag):    
    X_data = copy.copy(X_std)  
    
    # Rimappo le etichette per mostrare la legenda sul grafico finale
    X_data['quality'] = X_data['quality'].map(lambda x: 'Alta qualità' if x == 1 else 'Bassa qualità')   
    
    # Copio le etichette nel vettore y e le rimuovo da X
    y = X_data['quality']
    X_data = X_data.drop(columns=['quality'])
    
    # Calcolo le componenti principali
    sklearn_pca = sklearnPCA(n_components = components)
    Y_sklearn = sklearn_pca.fit_transform(X_data) 
    data = []
    colors = { 'red': '#2ca02c', 'green': '#d62728'}
    
    # Per ogni elemento costruisco una traccia dello scatter plot
    for name, col in zip(('Alta qualità', 'Bassa qualità'), colors.values()):
        trace = dict(
            type='scatter',
            x = Y_sklearn[y==name,0],
            y = Y_sklearn[y==name,1],
            mode = 'markers',
            name = name,
            marker = dict(
                color = col,
                size = 12,
                line = dict(
                    color='rgba(217, 217, 217, 0.14)',
                    width=0.5),
                opacity=0.8)
        )
        data.append(trace)

    layout = dict(
            xaxis=dict(title='PC1', showline=True),
            yaxis=dict(title='PC2', showline=True)
    )
    if(print_flag):
        fig = go.Figure(data=data, layout=layout)
        iplot(fig)
            
    # Ritorno i dati riproiettati e l'oggetto PCA
    return Y_sklearn, sklearn_pca

Il grafico sottostante mostra la distribuzione dei punti sul piano di assi le prime due componenti principali. Come si può vedere la proiezione sulle prime due componenti non ci permette di separare i dati in cluster ben distinti. Se così fosse stato avremmo potuto utilizzare solamente le prime due componenti come dataset, riducendo lo sforzo computazionale dovuto al fatto di dover lavorare in uno spazio di dimensionalità molto superiore a due. Non essendoci stata utile la PCA per disciminare i due gruppi di dati dobbiamo valutare quante delle componenti trovate possano tornarci utili per l'analisi.

In [16]:
doPCA(df_norm, 2, True)
my_pca_data, my_pca_obj = doPCA(df_norm, 5, False)
my_total_pca_data, my_total_pca_obj = doPCA(df_norm, 8, False)

Per prendere questa decisione abbiamo bisogno di sapere la quantità di varianza che ciascuna componente, quando tenuta in considerazione, aggiunge alla varianza globale della riproiezione. Il grafico successivo mostra chiaramente quando stiamo cercando.

In [17]:
# Funzione che serve a rappresentare graficamente la quantità di varianza che ogni componente
# principale è in grado di raccogliere
def print_explained_variance(explained_variance):    
    cumulative_variance = []
    cumulative_variance.append(explained_variance[0])
    
    # Ogni cella dell'array contiene tutta la varianza raccolta finora + quella della relativa cella
    for i in range(1, 8):
        old = cumulative_variance[i-1]
        new = explained_variance[i]
        cumulative_variance.append(old + new)
    
    # Una traccia per la varianza di ogni componente, l'altra per la varianza cumulativa
    data = [go.Bar(
            x=['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11'],
            y=[explained_variance[0], explained_variance[1], explained_variance[2], 
               explained_variance[3], explained_variance[4], explained_variance[5],
               explained_variance[6], explained_variance[7]],
            name="Varianza spiegata"
    ), go.Bar(
            x=['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11'],
            y=cumulative_variance,
            name="Varianza cumulativa"
    )]
    layout = go.Layout(
        title='Varianza spiegata e cumulativa',
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig) 

In [18]:
print_explained_variance(my_total_pca_obj.explained_variance_ratio_)

Un interrogativo che potrebbe sorgere spontaneo è relativo alla quantità di varianza che siamo disposti a perdere per ottenere un miglioramento in termini prestazionali, dovuto alla riduzione della dimensionalità del dataset. In letteratura non esiste una specifica soglia di separazione tra un buon valore di varianza ed uno mediocre o pessimo, anche se alcuni testi suggeriscono un valore che sia sopra a 85%. Seguendo dunque questo schema e volendo essere certi di avere un buon margine di varianza spiegata si è optato per selezionare le prime cinque componenti, che come è possibile verificare dal grafico soprastante spiegano una varianza che raggiunge il 90%. È certamente un ottimo compromesso.

<hr><hr>
<h2 align="center">Overfitting e valutazione delle prestazioni del modello</h2>

Costruire un modello utilizzando un unico set di training è un errore che va evitato, perchè esso apprenderà in maniera pressochè perfetta tutto ciò che riguarda tale set ma le sue prestazioni su nuovi dati potrebbero essere disastrose. Questa condizione è chiamata <b>overfitting</b> ed andrebbe evitata se si vuole costruire un buon classificatore. Per fare ciò si possono utilizzare diverse tecniche atte ad allenare il modello utilizzando diversi set di training, in modo tale che il modello non si "abitui" ad uno specifico set ma sia in grado di generalizzare il più possibile con lo scopo di classificare meglio i dati di test, quelli che non sono stati usati per l'apprendimento.<br>
In questa analisi sarà utilizzata la tecnica di validazione incrociata chiamata <b>k-Fold</b> che svolge le seguenti operazioni:
<ul>
    <li>il dataset viene suddiviso in k diversi subset</li>
    <li>Per ciascuno dei k subset:</li>
    <ul>
        <li>il modello viene allenato utilizzando i k-1 rimanenti subset</li>
        <li>il modello finale viene testato sulla rimanente porzione di dati (viene cioè calcolata l'accuratezza)</li>
    </ul>
</ul>
L'accuratezza complessiva del modello è data dalla media delle accuratezze ottenute per ciascuno dei passi descritti sopra.
<img src="img/grid_search_cross_validation.png" width="600">

<hr><hr>
<h2 align="center">Validazione del modello</h2>

Per ciascuno degli algoritmi di classificazione che useremo sarà quantificata la qualità della previsione utilizzando la metrica <b>F1-score</b>, definita come $$ F1_{score} = 2 \frac{\text{precisione} \times \text{sensibilità}}{\text{precisione} + \text{sensibilità}} $$

$$ \text{precisione} = \frac{T_{P}}{T_{P} + F_{P}} = \frac{Veri\ positivi}{Totale\ positivi\ previsti} $$
<br>
$$\text{sensibilità} = \frac{T_{P}}{T_{P} + F_{N}} = \frac{Veri\ positivi}{Totale\ positivi\ reali}$$

<ul>
    <li><b>TP</b>: Vero positivo (risultato corretto)</li>
    <li><b>TN</b>: Vero negativo (corretta assenza di risultato)</li>
    <li><b>FP</b>: Falso positivo (risultato inatteso)</li>
    <li><b>FN</b>: Falso negativo (risultato mancante)</li>
</ul><br>

La <b>precisione</b> è il rapporto tra le osservazioni predette in maniera corretta rispetto alle osservazioni totali predette come positive. Un'alta precisione significa che il modello ha predetto poche volte la classe positiva quando la classe giusta era quella negativa (basso valore di falsi positivi).<br><br>
La <b>sensibilità</b> è il rapporto tra le osservazioni correttamente classificate come positive rispetto a tutte le osservazioni che fanno parte di tale categoria. Essa rappresenta intuitivamente l'abilità del classificatore nel trovare tutte le osservazioni della classe positiva.<br><br>
Il valore <b>F1</b> calcola la media pesata tra precisione e sensibilità. Quindi esso tiene in considerazione sia i falsi positivi che che i falsi negativi.<br><br>
In un'analisi di questo tipo è bene decidere e quindi valutare quale peso dare agli errori. In altre parole dobbiamo chiederci se sia meglio ridurre al minimo il numero di esiti falsi positivi (e dunque avere alta precisione) oppure quello di falsi negativi (maggiore sensibilità). <br>
In questo caso potrebbe essere sensato <b>puntare sulla precisione</b>, in quanto distribuire un vino classificato come di alta qualità e poi ricevere lamentele da parte dei venditori (danno d'immagine) è sicuramente peggio che buttare del vino che si pensava essere di scarsa qualità quando era in realtà un buon vino (perlomeno per le grandi aziende, dove la quantità di prodotto è sicuramente alta). <br>


Sarà inoltre mostrata per ogni algoritmo la <b>curva ROC</b> e l'area sottesa dalla curva (<b>AUC</b>).<br>
La ROC è una curva di probabilità e AUC rappresenta il grado di misura della <b>separabilità</b>. In altre parole ci dice quanto il modello è abile a distinguere (in termini di <b>probabilità</b>) le due classi. Più è alto il valore di AUC e meglio il modello sarà in grado di prevedere 0 quando è 0 ed 1 quando è 1. In relazione a questa analisi, più è alto il valore di AUC e meglio il modello sarà in grado di classificare un vino come di alta qualità quando lo è effettivamente, e viceversa.
La curva ROC è rappresentata con i valori di <b>TPR (True Positive Rate)</b> sull'asse X e con i valori di <b>FPR (False Positive Rate)</b> sull'asse Y.<br>
Un modello eccellente ha un valore di AUC pari a 1, il che significa che ha un ottimo grado di separabilità. Viceversa un modello con AUC vicino allo zero non è assolutamente in grado di separare le due classi, e anzi classificherà in maniera inversa rispetto alla realtà. Infine quando AUC = 0.5 significa che il modello non ha la capacità di separare i dati delle due classi.<br>
Sotto sono mostrati degli esempi di curve ROC in base al valore di AUC.

<table>
    <tr>
        <td>
            <img src="img/roc_auc_1.png" width="300">
        </td>
        <td>
            <img src="img/roc_auc_1_1.png" width="150">
        </td>
        <td>
            <img src="img/roc_auc_0_7.png" width="300">
        </td>
        <td>
            <img src="img/roc_auc_0_7_1.png" width="150">
        </td>
    </tr>
    <tr>
        <td>
            <img src="img/roc_auc_0_5.png" width="300">
        </td>
        <td>
            <img src="img/roc_auc_0_5_1.png" width="150">
        </td>
        <td>
            <img src="img/roc_auc_0.png" width="300">
        </td>
        <td>
            <img src="img/roc_auc_0_1.png" width="150">
        </td>
    </tr>
</table>





<hr><hr>
<h2 align="center">Classificazione</h2>

<i>Classificare</i> significa riconoscere e discriminare oggetti facenti parti di diverse categorie sulla base di informazioni che sono in nostro possesso. Lo scopo è quello di trovare dei pattern nei dati che siano tipici di ciscuna categoria del dataset. A differenza della regressione, che mira a trovare un output di tipo continuo, la classificazione tenta di etichettare ciascun oggetto utilizzando solamente valori discreti (ad esempio dato un cane determinare la sua razza di appartenenza; le razze sono categorie, non possono venire espresse per mezzo di valori continui).
I modelli costruiti per classificare stanno vivendo un'epoca d'oro negli ultimi decenni, perchè sempre più le decisioni importanti vengono prese da modelli che basano le proprie scelte sulla storia passata, su quanto le decisioni passate in una o nell'altra direzione hanno portato benefici o creato problemi.<br>
Questo capitolo è dedicato interamente all'analisi del dataset utilizzando diversi algorimi di classificazione:<br>
<ul>    
    <li><b>Classificatore Bayesiano</b></li>
    <li><b>K-Nearest Neighbors</b></li>    
    <li><b>Support Vector Machine con kernel lineare</b></li>
    <li><b>Support Vector Machine con RBF kernel</b></li>    
    <li><b>Regressione Logistica</b></li>
    <li><b>Albero di decisione</b></li>
    <li><b>Random Forest</b></li>
</ul>
Per ciascuno di essi sarà fornita un'adeguata spiegazione teorica al fine di mettere il lettore nelle condizioni, seppur il linguaggio matematico possa essere a volte molto insistente, nelle condizioni di capire come funzioni un determinato algoritmo e perchè vengono trovati determinati valori.

In [19]:
# Questa classe gestisce e mantiene al suo interno tutte le informazioni sui classificatori e sui risultati, in modo
# da avere un unico contenitore di dati, grafici e risultati.
# Per eseguire un confronto sensato tra i classificatori, il dataset sarà
class Multiclassifier:
    def __init__(self, df, df_norm):
        # Etichette
        self.y = df['quality'].values

        # Elimino la colonna delle etichette dal dataframe
        self.X = copy.copy(df_norm).drop(columns=['quality']).values
        
        # Creo il dizionario dei colori
        self.colors_dict =	{
          "grey": "rgba(233, 233, 233 ,.8)",
          "orange": "rgba(255, 127, 14,.8)",
          "blue":  "rgba(31,119,180,.8)",
          "green": "rgba(0,204,0,.8)",
          "purple": "rgba(255,0,127,.8)",
          "black": "rgba(0,0,0,.8)",
          "azure": "rgba(0,255,255,.8)",
          "yellow": "rgba(241,250,66,.8)"
        }
                
        # Splitto il dataset in due subset: training and testing set
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.2, random_state = 19) 
        
        # Creo un array per contenere tutte le informazioni delle analisi e aggiungo la linea tratteggiata        
        self.roc_traces = []        
    
    # Metodo che aggiunge le informazioni di analisi all'array definito sopra
    def addScores(self, method_name, roc_test_trace, auc_score, color_str):
        
        # Cambio alcuni parametri della traccia
        roc_test_trace['marker']['color'] = self.colors_dict[color_str]
        roc_test_trace['name'] = method_name + " (AUC = " + str(round(auc_score,3)) + ")"        
        
        # Appendo all'array
        self.roc_traces.append(roc_test_trace) 
    
    # Metodo per disegnare la curva ROC comparativa finale
    def plotComparativeROC(self):
        # Aggiungo la traccia della linea tratteggiata
        trace_dash_line = go.Scatter(x=[0, 1], y=[0, 1], 
            mode='lines', 
            line=dict(color='navy', width=2, dash='dash'),
            showlegend=False)
        self.roc_traces.append(trace_dash_line)
        
        # Plotto le curve ROC
        self.plot_roc_curves(self.roc_traces, "Curva ROC comparativa di tutti i modelli")
                
        
    # Metodo per costruire le tracce della curva ROC
    def build_roc_trace(self, fpr, tpr, color, trace_name):                        
        # Singola traccia di curva PR
        trace = go.Scatter(x = fpr, y = tpr, mode = 'lines', name = trace_name, marker = dict(
            size = 10, color = self.colors_dict[color],
            line = dict(
            width = 2,
            color = 'rgb(0, 0, 0)')))
        
        # Linea tratteggiata
        trace_dash_line = go.Scatter(x=[0, 1], y=[0, 1], 
            mode='lines', 
            line=dict(color='navy', width=2, dash='dash'),
            showlegend=False)
        return trace, trace_dash_line
    
    # Metodo per plottare le curve ROC costruite
    def plot_roc_curves(self, roc_traces_list, title_str):
        layout_roc = dict(
                title = title_str,
                xaxis = dict(title = "FPR"),
                yaxis = dict(title = "TPR"),
                showlegend=True)
        fig_roc = go.Figure(data=roc_traces_list, layout=layout_roc)
        iplot(fig_roc)    
    
    # Metodo per rappresentare la matrice di confusione
    def plotConfusionMatrix(self, scores, supporto_0, supporto_1):        
        tn, fp, fn, tp = scores
        tn = round(tn/supporto_0, 3)
        fp = round(fp/supporto_0, 3)
        fn = round(fn/supporto_1, 3)
        tp = round(tp/supporto_1, 3)
        
        recall = tp/(tp+fn)
        specificity = 1 - fp/(fp+tn)
        balanced_accuracy = 0.5 * (recall + specificity)
        print("Specificità su testing set: %0.3f" % specificity)
        print("Balanced accuracy su testing set: %0.3f" % balanced_accuracy)

        trace = go.Heatmap(z=[[tn, fp], [fn, tp]],
                           x=["Bassa qualità", "Alta qualità"],
                           y=["Bassa qualità", "Alta qualità"]) 
        layout = dict(
              title="Matrice di confusione",
              yaxis = dict(title="Classe reale"),
              xaxis = dict(title="Classe predetta",)
             )
        figure = go.Figure(data=[trace], layout=layout)
        iplot(figure)
    
    # Metodo per disegnare i barplot di f1, precisione e recall
    def printScores(self, training_report, test_report):
        f1_train = training_report["macro avg"]['f1-score']
        precisione_train = training_report["macro avg"]["precision"]
        recall_train = training_report["macro avg"]["recall"]
        
        f1_test = test_report["macro avg"]['f1-score']
        precisione_test = test_report["macro avg"]["precision"]
        recall_test = test_report["macro avg"]["recall"]
        
        scores = [['F1', f1_train, f1_test], \
                  ['Precisione', precisione_train, precisione_test], \
                  ['Sensibilità', recall_train, recall_test]] 
  
        # Dataframe Pandas 
        df = pd.DataFrame(scores, columns = ['Score', 'Training set', 'Testing set'])                
        display(df)
    
    def doAnalysis(self, parameters, clf, clf_id):
        # Costruiamo la griglia per fare tuning dei parametri
        # Vogliamo il miglior modello in termini di balanced accuracy
        grid = GridSearchCV(clf, parameters, scoring='balanced_accuracy')
        
        # Nomi delle etichette
        target_names = ['Bassa qualità', 'Alta qualità']
        
        # Facciamo tuning dei parametri passando il dataset di training
        grid.fit(self.X_train, self.y_train)
        
        # Parametri del miglior modello trovato
        print("Parametri del miglior classificatore trovato: {}".format(grid.best_params_))               
        
        # Costruiamo un classificatore utilizzando i migliori parametri trovati
        
        # Classificatore Bayesiano
        if clf_id == 0:            
            best_clf = GaussianNB(var_smoothing = grid.best_params_['var_smoothing'])        
        
        # KNN
        if clf_id == 1:            
            best_clf = KNeighborsClassifier(n_neighbors = grid.best_params_['n_neighbors'], weights = grid.best_params_['weights'], \
                                       p = grid.best_params_['p'])
        # SVM Lineare
        elif clf_id == 2:
            best_clf = svm.SVC(C = grid.best_params_['C'], kernel='linear', gamma='auto', probability=True, \
                              class_weight = grid.best_params_['class_weight'])
        
        # SVM con kernel rbf
        elif clf_id == 3:
            best_clf = svm.SVC(kernel='rbf', C = grid.best_params_['C'], gamma = grid.best_params_['gamma'], probability=True, \
                              class_weight = grid.best_params_['class_weight'])
            
        
        # Regressione Logistica
        elif clf_id == 4:
            best_clf = LogisticRegression(penalty = grid.best_params_['penalty'], class_weight = grid.best_params_['class_weight'], \
                                         tol = grid.best_params_['tol'])
        
        # Albero di decisione
        elif clf_id == 5:
            best_clf = DecisionTreeClassifier(criterion = grid.best_params_['criterion'], \
                                             class_weight = grid.best_params_['class_weight'], \
                                             splitter = grid.best_params_['splitter'], \
                                             presort = grid.best_params_['presort'])
            
        # Random Forest
        elif clf_id == 6:            
            best_clf = RandomForestClassifier(n_estimators = grid.best_params_['n_estimators'], \
                                             criterion = grid.best_params_['criterion'], \
                                             class_weight = grid.best_params_['class_weight'], \
                                             bootstrap = grid.best_params_['bootstrap'])
            
                        

        # Fitto i dati di training utilizzando il miglior classificatore
        best_clf.fit(self.X_train, self.y_train)
        
        # Calcolo ed estraggo i risultati ottenuti con il miglior classificatore trovato ed utilizzato sul dataset di training
        # Stampo solo gli score inerenti la classe 1
        y_train_predict = best_clf.predict(self.X_train)
        training_report = classification_report(self.y_train, y_train_predict, output_dict=True, target_names = target_names)                
        
        # Calcolo le previsione sul set di testing          
        y_test_predict = best_clf.predict(self.X_test)                
        
        # Stampo i risultati ottenuti        
        test_report = classification_report(self.y_test, y_test_predict, output_dict=True, target_names = target_names)         
        supporto_0 = test_report["Bassa qualità"]["support"]
        supporto_1 = test_report["Alta qualità"]["support"]         
        
        # Plotto gli andamenti di f1, precisione e recall
        self.printScores(training_report, test_report)
        
        # Matrice di confusione
        #print(confusion_matrix(self.y_test, y_test_predict))
        print("Numero di elementi di classe 0 nel testing set: {}".format(supporto_0))
        print("Numero di elementi di classe 1 nel testing set: {}".format(supporto_1))
        self.plotConfusionMatrix(confusion_matrix(self.y_test, y_test_predict).ravel(), supporto_0, supporto_1)
        
        # Calcolo gli score per la curva ROC
        y_score_train = best_clf.predict_proba(self.X_train)
        y_score_test = best_clf.predict_proba(self.X_test)
        
        # Calcolo valori tpr, fpr per curva ROC
        fpr_train, tpr_train, _ = roc_curve(self.y_train, y_score_train[:,1])
        fpr_test, tpr_test, _ = roc_curve(self.y_test, y_score_test[:, 1])
        
        # Calcolo area sottesa dalla curva ROC per training e testing set
        auc_train = roc_auc_score(self.y_train, y_score_train[:, 1])
        auc_test = roc_auc_score(self.y_test, y_score_test[:, 1]) 
        
        # Costruisco le due tracce per il grafico della curva ROC
        trace_train, trace_dash_line = self.build_roc_trace(fpr_train, tpr_train, "blue", "Training set")
        trace_test, trace_dash_line = self.build_roc_trace(fpr_test, tpr_test, "orange", "Testing set")
        
        # Plotto le curve ROC
        self.plot_roc_curves([trace_train, trace_test, trace_dash_line], "Curva ROC di training e testing set")
        
        
        # Stampo il valore dell'area sotto la curva
        auc_string_train = "AUC del training set: %0.3f" % auc_train
        auc_string_test = "AUC del testing set: %0.3f" % auc_test        
        print(auc_string_train)
        print(auc_string_test)
        
        return trace_test, auc_test
    
    # Metodo per l'analisi tramite Naive Bayes classifier
    def NB_analysis(self):
        clf = GaussianNB()
       
        # Parametri del classificatore
        parameters = {
            'var_smoothing': (1e-7, 1e-8, 1e-9, 3e-9, 7e-9)
        }
        
        trace, auc = self.doAnalysis(parameters, clf, 0)                
        self.addScores("Bayes Classifier Gaussiano", trace, auc, "yellow") 

    # Metodo per l'analisi del dataset usando l'algoritmo KNN
    def KNN_analysis(self, param_range): 
        clf = KNeighborsClassifier() 
        
        # Parametri del classificatore
        parameters = {
            # Valori di K passati come parametro
            'n_neighbors': param_range, 
            
            # Tipologia di classificatore KNN
            'weights' : ('uniform', 'distance'),
            
            # Il valore di p indica la metrica utilizzata per valutare la distanza
            # p = 1 L1, p = 2 L2
            'p': (1, 2, 3)
        }        
        trace, auc = self.doAnalysis(parameters, clf, 1)                
        self.addScores("KNN", trace, auc, "blue")                        
        
    
    # Metodo per la l'analisi del dataset usando l'algoritmo SVM con kernel lineare
    def SVM_linear_analysis(self, param_range):                
        clf = svm.SVC(kernel='linear', gamma='auto', probability=True)
        
        # Parametri del classificatore
        parameters = {
            # Valori di C passati come parametro
            'C': param_range,
            'class_weight': ('balanced', {0:1, 1:2}, {0:1, 1:3}, {0:1, 1:4}, {0:1, 1:5}, {0:1, 1:7}, {0:1, 1:10},
                            {0:1, 1:20}, {0:1, 1:50}, {0:1, 1:100})
        }        
        trace, auc = self.doAnalysis(parameters, clf, 2)
        self.addScores("SVM con kernel lineare", trace, auc, "orange")
        
    # Metodo utile alla costruzione del modello tramite utilizzo del kernel rbf 
    def SVM_rbf_analysis(self, c_range, gamma_range):
        clf = svm.SVC(kernel='rbf', gamma='auto', probability=True)
        
        # Parametri del classificatore
        parameters = {
            # Valori di C passati come parametro
            'C': c_range,
            'gamma': gamma_range,
            'class_weight': ('balanced', {0:1, 1:2}, {0:1, 1:3}, {0:1, 1:4}, {0:1, 1:5}, {0:1, 1:7}, {0:1, 1:10},
                            {0:1, 1:20}, {0:1, 1:50}, {0:1, 1:100})
        }        
        trace, auc = self.doAnalysis(parameters, clf, 3)
        self.addScores("SVM con kernel RBF", trace, auc, "green")
        
    # Applicazione della regressione logistica
    def performLogRegression(self):
        clf = LogisticRegression()  
        
        # Parametri del classificatore
        parameters = {            
            'penalty': ('l1', 'l2'),
            'class_weight': ('balanced', {0:1, 1:2}, {0:1, 1:3}, {0:1, 1:4}, {0:1, 1:5}, {0:1, 1:7}, {0:1, 1:10},
                            {0:1, 1:20}, {0:1, 1:50}, {0:1, 1:100}),
            'tol': (1e-4, 5e-5, 2e-4, 1e-5, 1e-6, 2e-6, 3e-4)
        }        
        trace, auc = self.doAnalysis(parameters, clf, 4)
        self.addScores("Regressione Logistica", trace, auc, "purple")
        
    # Metodo per costruire un classificatore basato sull'algoritmo random forest
    def treeClassifier(self):
        clf = DecisionTreeClassifier()
        
        parameters = {
            'criterion': ('gini', 'entropy'),
            'splitter': ('best', 'random'),
            'presort': (True, False),
            'class_weight': ('balanced', {0:1, 1:2}, {0:1, 1:3}, {0:1, 1:4}, {0:1, 1:5}, {0:1, 1:7}, {0:1, 1:10},
                            {0:1, 1:20}, {0:1, 1:50}, {0:1, 1:100})
            
        }        
        trace, auc = self.doAnalysis(parameters, clf, 5)
        self.addScores("Albero di decisione", trace, auc, "azure")
    
    # Metodo per costruire un classificatore basato sull'algoritmo random forest
    def performRandomForest(self):
        clf = RandomForestClassifier()
        
        parameters = {
            'n_estimators': (10, 20, 25, 30, 40, 100),
            'criterion': ('gini', 'entropy'),
            'class_weight': ('balanced', {0:1, 1:2}, {0:1, 1:3}, {0:1, 1:4}, {0:1, 1:5}, {0:1, 1:7}, {0:1, 1:10},
                            {0:1, 1:20}, {0:1, 1:50}, {0:1, 1:100}),
            'bootstrap': (True, False)
        }        
        trace, auc = self.doAnalysis(parameters, clf, 6)
        self.addScores("Random Forest", trace, auc, "black")                

In [20]:
my_multiclassifier = Multiclassifier(df, df_norm)

<hr>
<h3 align="center">Classificatore Bayesiano</h3>

I cosiddetti <i>metodi Naive Bayes</i>, letteralmente <i>metodi ingenui di Bayes</i> sono un insieme di algoritmi di tipo <b>supervisionato</b> che operano per mezzo del teorema di Bayes con l'assunzione di indipendenza condizionale tra ciascuna coppia di osservazioni (ossia, il valore di un'osservazione non è influenzato dal valore dell'altra). Il <b>teorema di Bayes</b> si può riassumere nella seguente equazione:

$$ \begin{align}\begin{aligned}P(y \mid x_1, \dots, x_n) \propto P(y) \prod_{i=1}^{n} P(x_i \mid y)\\\Downarrow\\\hat{y} = \arg\max_y P(y) \prod_{i=1}^{n} P(x_i \mid y),\end{aligned}\end{align} $$

dove $ y $ è l'output e $ x_1, ..., x_n $ sono le osservazioni. Questo algoritmo risulta semplice da implementare e comodo da utilizzare, perchè richiede pochi dati di training per funzionare in maniera discreta e non consuma troppe risorse computazionali.<br>
I classificatori basati su questo algoritmo si differenziano in base alla probabilità (stimata a priori) che l'analista assegna ai dati di input. Ciò significa che è possibile costruire un diverso classificatore assumendo che i dati che si sta andando a studiare siano distribuiti con una certa distribuzione che è compito di chi compie l'analisi comprendere. Nel contesto di questa analisi sarà utilizzatp un classificatore naive di Bayes con l'assunzione che le osservazioni siano distribuiti secondo una distrubuzione gaussiana. In altre parole il termine dentro la produttoria sarà $$ P(x_i \mid y) = \frac{1}{\sqrt{2\pi\sigma^2_y}} \exp\left(-\frac{(x_i - \mu_y)^2}{2\sigma^2_y}\right) $$

In [21]:
my_multiclassifier.NB_analysis()

Parametri del miglior classificatore trovato: {'var_smoothing': 1e-07}


,Score,Training set,Testing set
0,F1,0.698020,0.784378
1,Precisione,0.682410,0.774698
2,Sensibilità,0.720214,0.795494


Numero di elementi di classe 0 nel testing set: 268
Numero di elementi di classe 1 nel testing set: 52
Specificità su testing set: 0.918
Balanced accuracy su testing set: 0.796


AUC del training set: 0.849
AUC del testing set: 0.885


Il modello lavora in maniera ottimale nel classificare i vini di bassa qualità, con un tasso di previsioni corrette che supera il <b>90%</b>. Viceversa il modello fatica nel classificare i vini di alta qualità, i quali vengono marchiati come di scarsa qualità poco più di tre volte su dieci. Di fatto non possiamo parlare di cattivo risultato, considerando il numero di osservazioni a nostra disposizione, ma se stiamo cercando di costruire un modello che sia in grado di mediare gli errori del I tipo e quelli del II tipo dovremmo probabilmente puntare ad altri algoritmi. Se invece fossimo intenzionati a scartare i vini che risulterebbero di bassa qualità possiamo fare affidamento su questo modello.

<hr>
<h3 align="center">K-Nearest Neighbors (KNN)</h3>

Il KNN è un algoritmo utilizzato per costruire e classificare pattern di oggetti basandosi sulle caratteristiche di essi. Tra gli algoritmi utilizzati nel Machine Learning è quello più semplice da implementare e da comprendere ed il funzionamento è il seguente: i voti dei k vicini del campione determinano il gruppo a cui assegnare l'oggetto in esame. Per evitare situazioni di parità è bene scegliere un valore k che non sia un divisore del numero di gruppi distinti presenti nel dataset. Per k = 1 il campione viene assegnato al gruppo dell'oggetto a lui più vicino.
Per scegliere il valore di k più adeguato è necessario valutare la tipologia di dati che si sta esaminando e la quantità di essi. Infatti, scegliere un valore di k elevato garantisce una migliore resistenza al rumore ma rende meno risconoscibili i margini di separazione.
Saranno provati alcuni valori di K al fine di valuare quale fornisce il punteggio F1-score maggiore.

Sono sotto riportati gli andamenti di precisione, recall, F1-score e le curve ROC al variare di K.
Le curve ROC in grigio rappresentano il calcolo fatto per ciascuno dei fold e per ogni valore di K, mentre in blu è rappresentata la curva che approssima tutte le altre.

In [22]:
# Uso il KNN testando tutti i valori di K compresi tra 1 e 40 con passo 1
my_multiclassifier.KNN_analysis(range(1, 40, 1))

Parametri del miglior classificatore trovato: {'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}


,Score,Training set,Testing set
0,F1,1.0,0.766444
1,Precisione,1.0,0.764434
2,Sensibilità,1.0,0.768513


Numero di elementi di classe 0 nel testing set: 268
Numero di elementi di classe 1 nel testing set: 52
Specificità su testing set: 0.922
Balanced accuracy su testing set: 0.768


AUC del training set: 1.000
AUC del testing set: 0.769


Gli score ottenuti ci mostrano come il modello sia andato in overfitting durante la fase di training, mostrando punteggi globali più bassi durante la fase di testing.<br>
Possiamo dire che il modello lavora discretamente bene nelle situazioni di previsione della classe positiva, in quanto riesce a mantenere <b>basso</b> il <b>numero di falsi positivi (7% delle previsioni)</b> e <b>moderatamente alto il numero dei veri positivi (61.5% delle previsioni)</b>.<br>
Ciò che paghiamo per avere questi risultati è il <b>piuttosto alto numero di falsi negativi</b>, che si attesta <b>intorno al 39%</b> delle previsioni. Tale valore potrebbe risultare estremamente alto in un contesto reale perchè l'impiego di questo modello potrebbe metterebbe a rischio l'economia di diversi produttori di questo vino, che potrebbero vedere il proprio vino perdere di valore nonostante sia di alta qualità.<br>
D'altra parte, questo modello potrebbe essere usato per perseguire una ricerca del vino di bassa qualità per un qualche scopo che esula da questa analisi: il livello di previsioni <b>vere negative</b> è infatti pari al <b>93%</b>.
La curva ROC ci mostra quanto detto, mettendo in risalto il fatto che al variare della soglia decisionale il modello riesca a mantenere un livello discretto di accuratezza.

<hr>
<h3 align="center">Support Vector Machine (SVM)</h3>

L'SVM è un algoritmo supervisionato per classificare elementi appartenenti a due o più classi. L'obiettivo è quello di trovare l'iperpiano che sia in grado di separare i dati nel modo migliore. Questo significa assegnare, per ciascun punto, un etichetta in base a dove il punto si trovi rispetto all'iperpiano di separazione.
Per trovare tale iperpiano abbiamo bisogno di risolvere un problema di ottimizzazione.


<img src="img/svm_1.png" width="400">

<hr>
<h4 align="center">Il problema di ottimizzazione</h4>

Se i dati sono perfettamente separabili nello spazio allora è dimostrato come sia possibile trovare l'iperpiano di separazione in un numero finito di passi. Ciò è garantito dal <b>teorema di convergenza del Perceptron</b>.<br>
A questo punto viene naturale chiedersi quale dei possibili iperpiani dobbiamo scegliere per separare i dati secondo le etichette. L'SVM per definizione ricerca l'iperpiano che massimizza la distanza tra i due gruppi di elementi (si veda l'immagine sopra).<br>
$$ \text{ Formulazione primale: } \min\frac{1}{2}||\omega||^2 \text{ subject to: } y^{(t)}(\omega \cdot x^{(t)} + b) \geq k, \ t=1,...,n $$ <br>
Si tratta di un <b>problema di ottimizzazione quadratico e con vincolo</b>. Esso può essere risolto utilizzando il metodo dei <b>moltiplicatori di Lagrange</b>, ed essendo un problema quadratico si tratta di trovare il minimo globale (unico) di una superficie paraboloidale.
Calcolando le derivate parziali della formulazione primale otteniamo la formulazione alternativa del problema. Essa è definita come $$ \text{ Formulazione alternativa: } \max\text{-}\frac{1}{2}\sum_{ij}{\alpha_i\alpha_j y_i y_j \vec{x_i} \cdot \vec{x_j}} + \sum_i{\alpha_i} \text{ subject to: } \sum_i{\alpha_i y_i} = 0 \text{, } \alpha_i > 0 $$

Formulando il problema in questo modo siamo in grado di calcolare la soluzione elimando la dipendenza dal vettore omega dei pesi e dal bias b. 
L'unica dipendenza rimasta per la risoluzione è quella relativa ai coefficienti alfa.
Questi coefficienti sono per lo più uguali a zero. Gli unici diversi da zero sono quelli relativi agli elementi presenti sui margini, che prendono il nome di <b>support vectors</b>, da cui il nome dell'algoritmo.

Nel caso in cui i dati non siano separabili linearmente, come in questo caso, possiamo pensare di risolvere il problema di ottimizzazione permettendo ad alcuni elementi del dataset di essere dalla parte sbagliata del margine, o addirittura dalla parte sbagliata dell'iperpiano. Questo si formalizza aggiungengo un ulteriore vincolo alle formulazioni precedenti in modo tale da limitare superiormente il limite d'errore a cui il modello può arrivare nella fase di classificazione.
Indichiamo tale valore con <b>C</b>.

<hr>
<h4 align="center">SVM Lineare</h4>

In questa prima applicazione dell'SVM utilizziamo un kernel lineare, ossia cerchiamo quell'iperpiano che massimizza la distanza tra i due cluster di dati. Non siamo in grado, a priori, di determinare quale sia il valore C che massimizza l'accuratezza sul set di testing. Per trovarlo possiamo creare un modello per un insieme di tali valori e scegliere il migliore osservando i risultati ottenuti. <br>
I valori di C con cui sarà testata l'accuratezza sono <b>{0.001, 0.01, 0.1, 1, 10, 1000, 10000}</b>.

In [23]:
my_multiclassifier.SVM_linear_analysis((0.07,0.08,0.09,0.1,0.2,0.22,0.25,0.3,0.4))

Parametri del miglior classificatore trovato: {'C': 0.3, 'class_weight': {0: 1, 1: 7}}


,Score,Training set,Testing set
0,F1,0.661788,0.728756
1,Precisione,0.651060,0.707265
2,Sensibilità,0.794304,0.834099


Numero di elementi di classe 0 nel testing set: 268
Numero di elementi di classe 1 nel testing set: 52
Specificità su testing set: 0.784
Balanced accuracy su testing set: 0.835


AUC del training set: 0.861
AUC del testing set: 0.916


Ciò che salta immediatamente all'occhio guardando la matrice di confusione è l'elevato valore di <b>veri positivi</b> che sono stati predetti nella fase di testing (<b>~89% delle previsioni</b>). Bisogna valutare se sia desiderabile o meno un valore di <b>falsi positivi che si attesti al 22%</b>. Ciò significa che per ogni 5 osservazioni di un vino di bassa qualità, una viene predetta come di alta qualità. Potrebbe in effetti essere un buon compromesso considerando il già indicato valori di veri positivi.<br>
Considerando anche che il numero di <b>falsi negativi</b> registrato è <b>piuttosto basso (11.5%)</b> e che quello di <b>veri negativi</b> è <b>sufficientemente alto (78.4%)</b> possiamo dire che il modello trovato riesce a mediare piuttosto bene tutte le possibili situazioni. Ne è dimostrazione la curva ROC, la cui area sottesa raggiunge un valore del 91.6%.

<hr>
<h4 align="center">SVM con kernel RBF</h4>

Quando i dati non sono linearmente separabili (come in questo caso) è più utile trovare un iperpiano che separi i dati in uno spazio di dimensione superiore. Facendo ciò si aumenta la complessità del calcolo del margine ma si riesce ad aumentare anche lo score del modello.<br>
Uno dei kernel maggiormente utilizzati nell'ambito del SVM è il cosiddetto <b>Radial basis function kernel (RBF Kernel)</b>. La formulazione standard del kernel è $$ K(x,y) = exp(-\frac{\|x - y\|^2}{2\sigma^2}) $$ che possiamo riscrivere come $$ K(x,y) = exp(-\gamma\|x - y\|^2), \quad \gamma = \frac{1}{2\sigma^2} $$

Questo tipo di kernel, come si può vedere dalla formula sopra, è formato da due diversi parametri su cui è necessario fare tuning: C e gamma.<br>

In [24]:
my_multiclassifier.SVM_rbf_analysis((0.07,0.08,0.09,0.1,0.2,0.3,0.4),\
                                    (0.1,0.2,0.3,0.4,0.5,0.6,0.65,0.7,0.75,0.8,0.9,1.0))

Parametri del miglior classificatore trovato: {'C': 0.3, 'class_weight': {0: 1, 1: 10}, 'gamma': 0.75}


,Score,Training set,Testing set
0,F1,0.626394,0.702937
1,Precisione,0.637711,0.694942
2,Sensibilità,0.791393,0.836825


Numero di elementi di classe 0 nel testing set: 268
Numero di elementi di classe 1 nel testing set: 52
Specificità su testing set: 0.731
Balanced accuracy su testing set: 0.837


AUC del training set: 0.871
AUC del testing set: 0.920


A fronte di un costo computazionale maggiore si ottiene un'accuratezza decisamente maggiore rispetto a quella ottenuta con un kernel lineare o utilizzando l'algoritmo KNN.

<hr>
<h3 align="center">Regressione logistica</h3>

Nonostante il nome possa far pensare ad un algoritmo di regressione, la regressione logistica è un metodo utilizzato per la classificazione, e deve la sua somiglianza lessicale con la regressione lineare ad una connessione storica con quella lineare.<br>
La regressione logistica si pone 
$$ h_ \theta (x) =  \frac{\mathrm{1} }{\mathrm{1} + e^- \theta^Tx },\ \theta= [b, \omega_1, \omega_2, \cdots \omega_d]$$ 
<img src="img/Logistic_curve.svg" width="600">

<hr>
<h4 align="center">Maximum Likelihood Estimation (MLE) e Gradient Ascent</h4>

Il modello che vogliamo costruire è un iperpiano che separa i dati in due gruppi e che denotiamo con la lettera $\theta$. La likelihood (probabilità) di un dato modello $\theta$ per un certo dataset $\vec{X}, \vec{y}$ è definito come $L(\theta) = P(\vec{y}|\vec{X}; \theta)$.<br>
La MLE ci dice che tra tutti i possibili iperpiani $\theta$ che possiamo trovare, noi cerchiamo quello che <b>massimizza</b> la probabilità per questi dati. Per fare ciò dobbiamo espandere la definizione di likelihood riportata sopra come $$L(\theta) = P(\vec{y}|\vec{X}; \theta) = \prod_i{P(y_i|x_i; \theta)}  $$
dato che la probabilità di un insieme di dati è uguale al prodotto delle probabilità dei singoli elementi.<br>
Quando y, cioè l'etichetta, è uguale a 1 allora $P(y_i|x_i; \theta) = h_ \theta (x)$.<br>
Viceversa quando y = 0 allora abbiamo che $P(y_i|x_i; \theta) = 1 - h_ \theta (x)$<br>
<br>Si ottiene quindi $$ L(\theta) = \prod_i{(1 - h(X_i;\theta))^{y_i}h(X_i;\theta)^{(1-y_i)}} $$


Passando in notazione logaritmica otteniamo 
$$ log L(\theta) = \sum_i{y_i  log(1 - h(X_i;\theta)) + (1 - y_i)log(h(X_i;\theta))}$$

L'obiettivo è trovare il valore di $\theta$ (l'iperpiano) per il quale la log-likelihood è massima rispetto a tutti gli altri iperpiani. Per fare ciò siamo soliti calcolare la derivata della quantità e poi porla uguale a zero. In questo caso non è possibile procedere così, perchè l'equazione è la somma di un numero arbitrario di termini e non ha una forma chiusa. Per trovare il massimo dell'equazione si utilizza la tecnica chiamata <b>gradient ascent</b> (che prende il nome di <i>gradient descent</i> quando si vuole risolvere un problema di minimizzazione). <br>
Essa consiste nell'aggiornare il valore di $\theta$ ad ogni iterazione, aggiungendo ad essa la derivata della funzione calcolata nel punto $\theta$. In questo modo si cerca di raggiungere il punto massimo della funzione per step successivi, senza passare dalla risoluzione analitica, che come abbiamo visto non è possibile fare.<br>
Questo aggiornamento si può scrivere matematicamente come
$$ \theta^{(t)} \rightarrow \theta^{(t-1)} + \eta f'(\theta) |_{\theta\ =\ \theta^{(t-1)}} $$
dove $\eta$ è un termine di regolazione chiamato <b>learning rate</b> e serve a quantificare la dimensione dello spostamento lungo la curva.<br><br>
Applicando il concetto di gradient ascent appena visto alla regressione logistica otteniamo
$$ \theta^{(t)} \rightarrow \theta^{(t-1)} + \eta \nabla log L(\theta) |_{\theta\ =\ \theta^{(t-1)}} $$
<br>
Una volta calcolati il vettore dei pesi $ \omega $ ed il bias, per classificare una nuova osservazione non dobbiamo fare altro che calcolare l'output della funzione sigmoide quando l'input è il valore delle feature dell'osservazione moltiplicata scalarmente con il vettore dei pesi. Un output sopra la soglia dello <b>0.5</b> classificherà l'osservazione con l'etichetta <b>1</b>, mentre uno sotto tale soglia sarà classificato con etichetta <b>0</b>. Il valore 0.5 si riferisce, come detto, ad una probabilità maggiore o minore il 50% che l'osservazione faccia parte della classe positiva.

L'ultimo concetto teorico che è bene citare riguarda la <b>regolarizzazione</b>. Utilizzando la procedura appena trattata c'è il rischio che il vettore $\omega$ dei pesi diventi estremamente grande, e questo porterebbe ad una situazione di overfitting in quanto la funzione sigmoide restituirebbe valori di output il più delle volte uguali a 0 od 1, e questa non è una situazione desiderabile. Per porre rimedio al problema è possibile <i>penalizzare</i> il modello ogni volta che sbaglia in modo da, appunto, regolarizzare il processo di apprendimento. Durante l'analisi con regressione logistica useremo sia la penalizzazione basata su norma L2 (chiamata <b>ridge</b>) che quella su norma L1 (anche detta <b>lasso</b>).

<hr>
<h4 align="center">Analisi dei dati usando la regressione logistica</h4>

In [25]:
my_multiclassifier.performLogRegression()

Parametri del miglior classificatore trovato: {'class_weight': {0: 1, 1: 10}, 'penalty': 'l1', 'tol': 0.0001}


,Score,Training set,Testing set
0,F1,0.631261,0.710112
1,Precisione,0.637519,0.695887
2,Sensibilità,0.786453,0.828789


Numero di elementi di classe 0 nel testing set: 268
Numero di elementi di classe 1 nel testing set: 52
Specificità su testing set: 0.754
Balanced accuracy su testing set: 0.829


AUC del training set: 0.864
AUC del testing set: 0.914


<hr>
<h3 align="center">Albero di decisione</h3>

Gli alberi di decisione sono algoritmi supervisionati e non parametrici usati sia per la regressione che per la classificazione. L'obiettivo è quello di creare un modello che sia in grado di assegnare le etichette ai dati sulla base di una regola di decisione dedotta dalle caratteristiche dei dati di training. Tra i vantaggi nell'utilizzo degli alberi di decisione troviamo:
<ul>
    <li>Sono molto semplici da visualizzare e da spiegare, anche a persone che non sono del mestiere</li>
    <li>Con gli alberi è possibile trattare dati categorici senza la necessità di creare variabili dummy.</li>
    <li>Richiedono una poco esosa preparazione dei dati</li>
</ul>
Essi portano con sè anche alcuni svantaggi:
<ul>
    <li>In genere gli alberi di decisione non forniscono un'accuratezza elevata come altri algoritmi classificativi.</li>
    <li>Gli alberi tendono ad essere poco robusti: un piccolo cambiamento nei dati può comportare un enorme modificazione dell'albero.</li>
    <li>I modelli basati su alberi di decisione tendono a fornire una non-buona generalizzazione dei dati, ossia vanno in overfitting.</li>
    <li>In presenza di dataset troppo sbilanciati verso una determinata classe il modello potrebbe risultare molto affetto da bias.</li>
</ul>

In questo paragrafo sarà costruito un modello basato su un albero decisione semplice, non utilizzando dunque alcuna tecnica di aggregazione degli alberi come per esempio il <b>Random Forest</b>, che sarà utilizzato nel prossimo paragrafo.

In [26]:
my_multiclassifier.treeClassifier()

Parametri del miglior classificatore trovato: {'class_weight': {0: 1, 1: 5}, 'criterion': 'entropy', 'presort': False, 'splitter': 'random'}


,Score,Training set,Testing set
0,F1,1.0,0.755051
1,Precisione,1.0,0.753127
2,Sensibilità,1.0,0.757032


Numero di elementi di classe 0 nel testing set: 268
Numero di elementi di classe 1 nel testing set: 52
Specificità su testing set: 0.918
Balanced accuracy su testing set: 0.757


AUC del training set: 1.000
AUC del testing set: 0.757


<hr>
<h3 align="center">Random Forest</h3>

Gli alberi di decisione appena trattati soffrono di <b>alta varianza</b>. In altre parole se noi dividessimo il training set in due parti in maniera casuale e costruissimo due modelli ad albero, noteremmo delle differenze in termini di accuratezza raggiunta. Di contro saremmo in grado di ottenere all'incirca gli stessi risultati su subset diversi abbassando la varianza della procedura. 

Il termine <b>bagging</b> significa <b>boostrap aggregation</b> ed è una procedura utilizzata allo scopo di ridurre la varianza del metodo utilizzato per l'analisi. Si basa sull'assunto che dato un insieme di variabili indipendenti ciascuna di variaza $ \sigma^2 $, la varianza media dell'insieme è data da $ \sigma^2/n $. Questo significa che applicare un operatore di media su un dataset ne abbassa la varianza. Di conseguenza un metodo naturale per ridurre la varianza è quello di prendere diversi set da una data popolazione, costruire un modello predittivo usando ciascun training set e calcolare la media delle accuratezze ottenute.

Fatte queste premesse possiamo definire il random forest come un metodo di classificazione basato sull'utilizzo di una moltitudine di alberi di decisione e la cui previsione si basa su quella che è stata la classe predetta il più delle volte dai singoli alberi da cui è composto.
La particolarità di questo classificatore risiede nella maniera con cui i nodi vengono splittati: invece di scegliere la miglior feature e dividere di conseguenza il nodo sulla base di essa viene scelta la migliore feature all'interno di un ristretto sottoinsieme di feature. Questo conduce ad un'ampia diversità tra i singoli alberi e di conseguenza ad un miglior modello finale.

In [27]:
my_multiclassifier.performRandomForest()

Parametri del miglior classificatore trovato: {'bootstrap': False, 'class_weight': {0: 1, 1: 5}, 'criterion': 'gini', 'n_estimators': 30}


,Score,Training set,Testing set
0,F1,1.0,0.789719
1,Precisione,1.0,0.865537
2,Sensibilità,1.0,0.748421


Numero di elementi di classe 0 nel testing set: 268
Numero di elementi di classe 1 nel testing set: 52
Specificità su testing set: 0.978
Balanced accuracy su testing set: 0.748


AUC del training set: 1.000
AUC del testing set: 0.957


<hr>
<h3 align="center">Confronto tra i modelli</h3>

Allo scopo di confrontare il comportamento generale dei cinque algoritmi di classificazione utilizzati in questa analisi, sono sotto riportate le curve ROC calcolate sul testing set.

In [28]:
my_multiclassifier.plotComparativeROC()

Come è possibile notare l'algoritmo che meglio si è comportato è stato il <b>Random Forest</b>, che ha registrato un'area sottostante la curva pari a circa il ~95%. Dall'altra parte troviamo il KNN con un valore di AUC pari a ~77%, che come visto in fase di analisi del modello si è comportato discretamente bene in tutte le previsioni senza tuttavia eccedere in nessuna delle metriche utilizzate.

<hr><hr>
<h2 align="center">Conclusioni</h2>

Per mezzo di questa analisi siamo riusciti a costruire un insieme di modelli in grado di prevedere la qualità del Vino <b>Vinho Verde</b> soltanto sulla base di otto sue caratteristiche ed ottenendo, almeno per alcuni dei modelli, risultati soddisfacenti.<br>
In questo particolare caso abbiamo immaginato uno scenario in cui prevedere un falso positivo fosse ben più grave che prevedere un falso negativo, perchè ci siamo posti nell'ottica di star sviluppando un prodotto ad uso commerciale (atto a fare investimenti commerciali). Abbiamo comunque tenuto in considerazione altri fattori, quali ad esempio un forte indebolimento del mercato locale del Vinho Verde nel caso di un elevato numero di falsi negativi, che in alcuni casi estremi avrebbe potuto portare piccole aziende vicine al collasso.